
# Setup of the environment



First, we are setting up our environment. We use an already compiled and packaged installation of HOOMD-blue and the DLExt plugin. We copy it from Google Drive and install pysages for it. This may require you to have read permissions to the shared Google Drive. We also have a Google Colab that performs this installation for reference.


In [1]:
%%bash

BASE_URL="https://drive.google.com/u/0/uc?id=1hsKkKtdxZTVfHKgqVF6qV2e-4SShmhr7&export=download"
wget -q --load-cookies /tmp/cookies.txt "$BASE_URL&confirm=$(wget -q --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate $BASE_URL -O- | sed -rn 's/.*confirm=(\w+).*/\1\n/p')" -O pysages-env.zip
rm -rf /tmp/cookies.txt

In [2]:
%env PYSAGES_ENV=/env/pysages

env: PYSAGES_ENV=/env/pysages


In [3]:
%%bash

mkdir -p $PYSAGES_ENV
unzip -qquo pysages-env.zip -d $PYSAGES_ENV
rm pysages-env.zip

In [4]:
!update-alternatives --auto libcudnn &> /dev/null

In [5]:
import os
import sys

ver = sys.version_info
sys.path.append(os.environ["PYSAGES_ENV"] + "/lib/python" + str(ver.major) + "." + str(ver.minor) + "/site-packages/")

os.environ["XLA_FLAGS"] = "--xla_gpu_strict_conv_algorithm_picker=false"
os.environ["LD_LIBRARY_PATH"] = "/usr/lib/x86_64-linux-gnu:" + os.environ["LD_LIBRARY_PATH"]


## PySAGES

The next step is to install PySAGES.
First, we install the jaxlib version that matches the CUDA installation of this Colab setup. See the JAX documentation [here](https://github.com/google/jax) for more details.


In [6]:
%%bash

pip install -q --upgrade pip
# Installs the wheel compatible with CUDA 11 and cuDNN 8.0.5.
pip install -q --upgrade "jax[cuda11_cudnn805]" -f https://storage.googleapis.com/jax-releases/jax_releases.html &> /dev/null


We test the jax installation and check the versions.



Now we can finally install PySAGES. We clone the newest version from [here](https://github.com/SSAGESLabs/PySAGES) and build the remaining pure python dependencies and PySAGES itself.


In [7]:
%%bash

rm -rf PySAGES
git clone https://github.com/SSAGESLabs/PySAGES.git &> /dev/null
cd PySAGES
pip install -q . &> /dev/null


# Dynamic Undocking simulation



The Dynamical Undocking method (DuCK)has been described as an
orthogonal method to docking for effective distinguishing true ligands from
decoys in binding affinity interactions with proteins.

For this Colab, we are readimg Amber files to import to OpenMM, for this, we use the ParMEd.

In [8]:
%%bash

cd /content

RST="https://github.com/SSAGESLabs/PySAGES/raw/miccom-ws/examples/openmm/duck/in-md2.rst"
PRMTOP="https://github.com/SSAGESLabs/PySAGES/raw/miccom-ws/examples/openmm/duck/in.prmtop"

wget -q $RST
wget -q $PRMTOP

In [14]:
!pip install -q parmed &> /dev/null


from pysages.methods import CVRestraints
from pysages.methods import ABF
from pysages.colvars import Distance
from pysages.utils import try_import

#from IPython import get_ipython
openmm = try_import("openmm", "simtk.openmm")
unit = try_import("openmm.unit", "simtk.unit")
app = try_import("openmm.app", "simtk.openmm.app")
from openmm import *
from openmm.app import *
from openmm.unit import *

# ParmEd Imports
from parmed import load_file, unit as u
from parmed.openmm import StateDataReporter, NetCDFReporter

import jax
import jax.numpy as np
import math
import numpy
import pysages
import matplotlib.pyplot as plt


pi = numpy.pi


def generate_simulation(
    T = 300.0 * kelvin,
    dt = 1.0 * femtoseconds
):
    print('Loading AMBER files...')
    ala2_solv = load_file('in.prmtop', 'in-md2.rst')
    system = ala2_solv.createSystem(nonbondedMethod=PME,rigidWater=True,switchDistance=1.0*nanometer,
        nonbondedCutoff=1.2*nanometer, constraints=None)
    # Create the integrator to do Langevin dynamics
    integrator = openmm.LangevinIntegrator(
        T,  # Temperature of heat bath
        1.0/picoseconds,  # Friction coefficient
        dt,  # Time step
    )
    # Define the platform to use; CUDA, OpenCL, CPU, or Reference. Or do not specify
    platform = Platform.getPlatformByName('CUDA')
    
    # Create the Simulation object
    sim = app.Simulation(ala2_solv.topology, system, integrator,platform)

    # Set the particle positions
    sim.context.setPositions(ala2_solv.positions)
    sim.reporters.append(StateDataReporter('data.txt', 1000, step=True,separator=' '))
    return sim


We define some helper functions for plotting and saving the data.

In [10]:
def plot_energy(result):
    energy = result["fes_fn"]
    mesh = numpy.asarray(result["mesh"])
    print(mesh.shape)
    A = energy(mesh)
    surface = A

    fig, ax = plt.subplots()

    ax.set_xlabel("CV")
    ax.set_ylabel("Free energy $[\\epsilon]$")

    ax.plot(mesh,surface, color="teal")

    fig.savefig("energy.png")


def plot_forces(result):
    fig, ax = plt.subplots()

    ax.set_xlabel("CV")
    ax.set_ylabel("Forces $[\\epsilon]$")

    forces = numpy.asarray(result["mean_force"])
    x = numpy.asarray(result["mesh"])
    ax.plot(x, forces, color="teal")

    fig.savefig("forces.png")


def plot_histogram(result):
    surface = numpy.asarray((result["histogram"]) / numpy.nanmax(numpy.asarray(result["histogram"])))

    fig, ax = plt.subplots()

    ax.set_xlabel("CV")
    ax.set_ylabel("Histogram $[\\epsilon]$")

    x = numpy.asarray(result["mesh"])
    ax.plot(x, surface, color="teal")

    fig.savefig("histogram.png")


def save_energy_forces(result):
    Energy = result["fes_fn"]
    Forces = numpy.asarray(result["mean_force"])
    Grid = numpy.asarray(result["mesh"])
    A = Energy(Grid)
    surface = A

    hist = numpy.asarray((result["histogram"]) / numpy.nanmax(numpy.asarray(result["histogram"])))
    numpy.savetxt("FES.csv", numpy.column_stack([Grid, surface]))
    numpy.savetxt("Forces.csv", numpy.column_stack([Grid, Forces]))
    numpy.savetxt("Histogram.csv", numpy.column_stack([Grid, hist]))


The next step is to select our orthogonal CV to charactarize the undocking.
In this case the distance of a pair of H-bond forming atoms will do.


In [11]:
indices=[[4668],[2501]]

cvs = (Distance(indices), )

grid = pysages.Grid(
    lower = (0.25,),
    upper = (0.50,),
    shape = (16,),
    periodic = False
)
topology = (8,8)


Now all that we have left is run the simulation and analyze the results.


In [ ]:
restraints = CVRestraints(lower=(0.23,), upper=(0.60,), kl=(10000.,), ku=(10000.,))
method = ABF(cvs, grid, restraints=restraints)
state = pysages.run(method, generate_simulation, 50000)

Loading AMBER files...


/usr/local/lib/python3.7/dist-packages/pysages/methods/abf.py:213: FutureWarning: The sym_pos argument to solve() is deprecated and will be removed in a future JAX release. Use assume_a='pos' instead.
  Wp = linalg.solve(Jxi @ Jxi.T, Jxi @ p, sym_pos="sym")
/usr/local/lib/python3.7/dist-packages/pysages/methods/abf.py:213: FutureWarning: The sym_pos argument to solve() is deprecated and will be removed in a future JAX release. Use assume_a='pos' instead.
  Wp = linalg.solve(Jxi @ Jxi.T, Jxi @ p, sym_pos="sym")
/usr/local/lib/python3.7/dist-packages/pysages/methods/abf.py:213: FutureWarning: The sym_pos argument to solve() is deprecated and will be removed in a future JAX release. Use assume_a='pos' instead.
  Wp = linalg.solve(Jxi @ Jxi.T, Jxi @ p, sym_pos="sym")



And plot the results.


In [ ]:
result = pysages.analyze(state, topology=topology)

In [ ]:
plot_energy(result)
plot_forces(result)
plot_histogram(result)
save_energy_forces(result)